# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 68 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/1 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2304.13046


extracting tarball to tmp_2304.13046...

 done.


list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.13046-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.13046) | **The Pristine Dwarf-Galaxy survey -- V. The edges of the dwarf galaxy  Hercules**  |
|| N. Longeard, et al. -- incl., <mark>K. Malhan</mark> |
|*Appeared on*| *2023-04-27*|
|*Comments*| **|
|**Abstract**| We present a new spectroscopic study of the dwarf galaxy Hercules (d ~ 132 kpc) with data from the Anglo-Australian Telescope and its AAOmega spectrograph together with the Two Degree Field multi-object system to solve the conundrum that whether Hercules is tidally disrupting. We combine broadband photometry, proper motions from Gaia, and our Pristine narrow-band and metallicity-sensitive photometry to efficiently weed out the Milky Way contamination. Such cleaning is particularly critical in this kinematic regime, as both the transverse and heliocentric velocities of Milky Way populations overlap with Hercules. Thanks to this method, three new member stars are identified, including one at almost 10rh of the satellite. All three have velocities and metallicities consistent with that of the main body. Combining this new dataset with the entire literature cleaned out from contamination shows that Hercules does not exhibit a velocity gradient (d<v>/dX = 0.1+0.4/-0.2 km s-1 arcmin-1) and, as such, does not show evidence to undergo tidal disruption. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2304.13046.md
    + _build/html/tmp_2304.13046/./cahk_density.png
    + _build/html/tmp_2304.13046/./field.png
    + _build/html/tmp_2304.13046/./Her_velocity_gradient_pdfs_noaat_vs_aat.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\fmmm}[1]{\mbox{#1}}$
$\newcommand{\scnd}{\mbox{\fmmm{"}\hskip-0.3em .}}$
$\newcommand{\scnp}{\mbox{\fmmm{"}}}$
$\newcommand{\mcnd}{\mbox{\fmmm{'}\hskip-0.3em .}}$
$\newcommand{\mnras}{MNRAS}$
$\newcommand{\pasa}{PASA}$
$\newcommand{\nat}{Nature}$
$\newcommand{\araa}{ARAA}$
$\newcommand{\aj}{AJ}$
$\newcommand{\apj}{ApJ}$
$\newcommand{\apjl}{ApJ}$
$\newcommand{\apjs}{ApJSupp}$
$\newcommand{\aap}{A\&A}$
$\newcommand{\aaps}{A\&ASupp}$
$\newcommand{\pasp}{PASP}$
$\newcommand{\pasj}{PASJ}$
$\newcommand{\arraystretch}{0.3}$
$\newcommand{\arraystretch}{0.3}$
$\newcommand{\arraystretch}{0.3}$
$\newcommand{\arraystretch}{0.3}$
$\newcommand{\ltsima}{\; \buildrel < \over \sim \;}$
$\newcommand{\lta}{\lower.5ex\hbox{\ltsima}}$
$\newcommand{\gtsima}{\; \buildrel > \over \sim \;}$
$\newcommand{\simgt}{\lower.5ex\hbox{\gtsima}}$
$\newcommand{\erf}{\mathop{\rm erf}\nolimits}$
$\newcommand{\sech}{ \mathop{\rm sech}\nolimits}$
$\newcommand{\csch}{ \mathop{\rm csch}\nolimits}$
$\newcommand{\arcsinh}{\mathop{\rm arcsinh}\nolimits}$
$\newcommand{\arccosh}{\mathop{\rm arccosh}\nolimits}$
$\newcommand{\arctanh}{\mathop{\rm arctanh}\nolimits}$
$\newcommand{\arccoth}{\mathop{\rm arccoth}\nolimits}$
$\newcommand{\arcsech}{\mathop{\rm arcsech}\nolimits}$
$\newcommand{\arccsch}{\mathop{\rm arccsch}\nolimits}$
$\newcommand{\arccot}{\mathop{\rm arccot}\nolimits}$
$\newcommand{\arcsec}{\mathop{\rm arcsec}\nolimits}$
$\newcommand{\arccsc}{\mathop{\rm arccsc}\nolimits}$
$\newcommand{\ylm}{\mathop{\rm Y}_l^m\nolimits}$
$\newcommand{\ylmp}{\mathop{\rm Y}_{l'}^{m'}\nolimits}$
$\newcommand{\real}{\Re e}$
$\newcommand{\imag}{\Im m}$
$\newcommand{\km}{{\rm km}}$
$\newcommand{\kms}{{\rm km \; s^{-1}}}$
$\newcommand{\mas}{{\rm mas}}$
$\newcommand{\masyr}{{\rm mas/yr}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand{\mpc}{{\rm Mpc}}$
$\newcommand{\msun}{{\rm M_\odot}}$
$\newcommand{\lsun}{{\rm L_\odot}}$
$\newcommand{\rsun}{{\rm R_\odot}}$
$\newcommand{\pc}{{\rm pc}}$
$\newcommand{\cm}{{\rm cm}}$
$\newcommand{\yr}{{\rm yr}}$
$\newcommand{\au}{{\rm AU}}$
$\newcommand{\g}{{\rm g}}$
$\newcommand{\om}{\Omega_0}$
$\newcommand{\}{ca}$
$\newcommand{\}{r}$
$\newcommand{\}{magnitude}$
$\newcommand{\kr}{{\cal K}_r}$
$\newcommand{\kz}{{\cal K}_z}$
$\newcommand{\kzz}{{\cal K}_z(z)}$
$\newcommand{\mss}{{\rm M}_\odot \rm pc^{-2}}$
$\newcommand{\msss}{{\rm M}_\odot \rm pc^{-3}}$
$\newcommand{\Aa}{\; \buildrel \circ \over{\rm A}}$
$\newcommand{Å}{\; \buildrel \circ \over{\rm A}}$
$\newcommand{\yr}{{\rm yr}}$
$\newcommand{\CompactFigs}{0}$
$\newcommand{\UseFigs}{1}$
$\newcommand{\deg}{^\circ}$
$\newcommand{\degg}{\hbox{\null^\circ\hskip-3pt .}}$
$\newcommand{\sec}{\hbox{"\hskip-3pt .}}$
$\newcommand{\half}{{\scriptstyle{1\over2}}}$
$\newcommand{\s}{\ifmmode \widetilde \else \~\fi}$
$\newcommand{\=}{\overline}$
$\newcommand{\scre}{{\cal E}}$
$\newcommand{\}{spose}$
$\newcommand{\larrow}{\leftarrow}$
$\newcommand{\rarrow}{\rightarrow}$
$\newcommand{\llangle}{\langle\langle}$
$\newcommand{\rrangle}{\rangle\rangle}$
$\newcommand{\etal}{{\it et al. }}$
$\newcommand{\cf}{{\it cf. }}$
$\newcommand{\eg}{{ e.g., }}$
$\newcommand{\ie}{{ i.e., }}$
$\newcommand{\lta}{\mathrel{\spose{\lower 3pt\hbox{\mathchar"218}}$
$     \raise 2.0pt\hbox{\mathchar"13C}}}$
$\newcommand{\gta}{\mathrel{\spose{\lower 3pt\hbox{\mathchar"218}}$
$     \raise 2.0pt\hbox{\mathchar"13E}}}$
$\newcommand{\Dt}{\spose{\raise 1.5ex\hbox{\hskip3pt\mathchar"201}}}$
$\newcommand{\dt}{\spose{\raise 1.0ex\hbox{\hskip2pt\mathchar"201}}}$
$\newcommand{\del}{\nabla}$
$\newcommand{\delv}{\bb\nabla}$
$\newcommand{\r}{{\rm r^{1/4}}}$
$\newcommand{\jla}{J_{\lambda}}$
$\newcommand{\jmu}{J_{\mu}}$
$\newcommand{\jnu}{J_{\nu}}$
$\newcommand{\pomega}{\varpi}$
$\newcommand{\sigla}{\sigma_{\lambda}}$
$\newcommand{\sigmu}{\sigma_{\mu}}$
$\newcommand{\signu}{\sigma_{\nu}}$
$\newcommand{\dotsfill}{\leaders\hbox to 1em{\hss.\hss}\hfill}$
$\newcommand{\sun}{\odot}$
$\newcommand{\earth}{\oplus}$
$\newcommand{\Gyr}{{\rm Gyr}}$
$\newcommand{\FeH}{{\rm[Fe/H]}}$
$\newcommand{\kmsd}{{\rm km/s/degree}}$
$\newcommand{\}{Ger}$
$\newcommand{\}{Rod}$
$\newcommand{\}{Mike}$
$\newcommand{\}{Nickl}$
$\newcommand{\}{Nick}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# The Pristine Dwarf-Galaxy survey - V. The edges of the dwarf galaxy Hercules

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2304.13046-b31b1b.svg)](https://arxiv.org/abs/2304.13046)<mark>Appeared on: 2023-04-27</mark> - 

</div>
<div id="authors">

N. Longeard, et al. -- incl., <mark>K. Malhan</mark>

</div>
<div id="abstract">

**Abstract:** We present a new spectroscopic study of the dwarf galaxy Hercules (d $\sim 132$ kpc) with data from the Anglo-Australian Telescope and its AAOmega spectrograph together with the Two Degree Field multi-object system to solve the conundrum that whether Hercules is tidally disrupting. We combine broadband photometry, proper motions from Gaia, and our Pristine narrow-band and metallicity-sensitive photometry to efficiently weed out the Milky Way contamination. Such cleaning is particularly critical in this kinematic regime, as both the transverse and heliocentric velocities of Milky Way populations overlap with Hercules. Thanks to this method, three new member stars are identified, including one at almost $10$ $r_h$ of the satellite. All three have velocities and metallicities consistent with that of the main body. Combining this new dataset with the entire literature cleaned out from contamination shows that Hercules does not exhibit a velocity gradient (d $\langle v \rangle$ /d $\chi$ $ = 0.1^{+0.4}_{-0.2}$ km s $^{-1}$ arcmin $^{-1}$ ) and, as such, does not show evidence to undergo tidal disruption.

</div>

<div id="div_fig1">

<img src="tmp_2304.13046/./cahk_density.png" alt="Fig7" width="100%"/>

**Figure 7. -** Pristine colour-colour space, with the ($g-i$)$_0$ temperature proxy and the $CaHK_0 - g_0 - 1.5$($g-i$)$_0$ colour on the y-axis. In this plot, [Fe/H] decreases as values on the y-axis decrease. The density plot of the MW contamination is also shown with the colour scale. The density decreases as the colour goes darker. The $1$, $2$ and $3$\s$igma$ contours are shown as white dotted lines. Literature members and non-members are shown as blue diamonds and crosses respectively and are only the ones with a spectroscopic metallicity and a proper motion measurement to ensure their membership status. Our new AAT members as shown as red circles, while uncertain candidates are represented as orange diamonds. (*cahk_density*)

</div>
<div id="div_fig2">

<img src="tmp_2304.13046/./field.png" alt="Fig10" width="100%"/>

**Figure 10. -** _Left panel:_ Spatial distribution of the AAT spectroscopic sample. Newly discovered members are shown as red circles, while uncertain candidates are shown as orange diamonds. Non-members from the AAT sample are shown as red crosses. Previously known members from the literature are represented as smaller blue diamonds.  The two half-light radii of Hercules as inferred by \citet[M18]{munoz18} are shown as a purple ellipse. _Right panel:_ CMD of our spectroscopic sample superimposed with a metal-poor Darmouth isochrone at the distance of Hercules. (*field*)

</div>
<div id="div_fig3">

<img src="tmp_2304.13046/./Her_velocity_gradient_pdfs_noaat_vs_aat.png" alt="Fig9" width="100%"/>

**Figure 9. -** PDFs of the velocity gradient of Hercules in the three cases detailed in section 5. (*pdfs_vel_grad*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2304.13046"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

196  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
